# Code for iteration 2, addressing categorical data

In [1]:
# do the imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
import statsmodels.api as sm 
from sklearn.metrics import mean_squared_error

pd.options.display.max_rows = 5000

//anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
# get the original data post data cleaning in data clean jupyter notebook and add changes from iter 1
df_clean = pd.read_csv('../datasets/cleaned_up_data.csv')


In [3]:
# get the iteration 1 chosen features df
df_iter_1 = pd.read_csv('../datasets/iter_1_df.csv')
df_iter_1 = df_iter_1.drop('Unnamed: 0', axis= 1)

In [4]:
# dummy up the selected categorical features off the original clean up data
df_neighborhood_dummies = pd.get_dummies(df_clean['Neighborhood'], drop_first= True)

df_bldg_type_dummies = pd.get_dummies(df_clean['Bldg_Type'], drop_first= True)

df_house_style_dummies = pd.get_dummies(df_clean['House_Style'], drop_first= True)

def month_to_season_conversion(month):
    if month <= 3:
        return 'spring'
    elif month <= 6:
        return 'summer'
    elif month <= 9:
        return 'fall'
    elif month <= 12:
        return 'winter'
    else:
        print('Houston, we have a problem!')
df_clean['season'] = df_clean['Mo_Sold'].apply(month_to_season_conversion)
df_season_dummies = pd.get_dummies(df_clean['season'], drop_first= True)

df_sale_type_dummies = pd.get_dummies(df_clean['Sale_Type'], drop_first= True)


In [5]:
df_clean = pd.concat([df_clean, df_neighborhood_dummies, df_bldg_type_dummies, df_house_style_dummies, \
               df_season_dummies, df_sale_type_dummies], axis= 1)

In [6]:
df_clean.corr()['SalePrice']

Unnamed: 0                0.027433
Id                       -0.053056
PID                      -0.253634
MS_SubClass              -0.089106
Lot_Frontage              0.186822
Lot_Area                  0.305112
Overall_Qual              0.802675
Overall_Cond             -0.101462
Year_Built                0.571737
Year_Remod/Add            0.550175
Mas_Vnr_Area              0.511303
BsmtFin_SF_1              0.445234
BsmtFin_SF_2              0.015830
Bsmt_Unf_SF               0.190318
Total_Bsmt_SF             0.664222
1st_Flr_SF                0.647931
2nd_Flr_SF                0.247839
Low_Qual_Fin_SF          -0.041925
Gr_Liv_Area               0.718762
Bsmt_Full_Bath            0.283454
Bsmt_Half_Bath           -0.046159
Full_Bath                 0.537521
Half_Bath                 0.282535
Bedroom_AbvGr             0.135393
Kitchen_AbvGr            -0.126179
TotRms_AbvGrd             0.508719
Fireplaces                0.474690
Garage_Yr_Blt             0.259601
Garage_Cars         

In [7]:
iter_1_features = ['PID', 'MS_SubClass','Lot_Frontage','Lot_Area','Overall_Qual','Overall_Cond','Year_Built', 
            'Year_Remod/Add','Mas_Vnr_Area','BsmtFin_SF_1','Total_Bsmt_SF','1st_Flr_SF','2nd_Flr_SF', 'Gr_Liv_Area',
            'Bsmt_Full_Bath','Full_Bath', 'Half_Bath','Bedroom_AbvGr','TotRms_AbvGrd', 'Fireplaces','Garage_Yr_Blt',
            'Garage_Cars', 'Garage_Area', 'Wood_Deck_SF', 'Open_Porch_SF','Enclosed_Porch','Bsmt_Qual_missing',
            'Garage_Type_missing']

In [8]:
additional_features_iter_2 = [ 'BrDale', 'BrkSide', 'Edwards', 
                   'IDOTRR', 'MeadowV', 'NAmes', 'NoRidge', 'NridgHt','OldTown', 'Sawyer', 'Somerst',
                   'StoneBr', 'Timber', 'Veenker', '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE', '1.5Unf','2Story',
                   'ConLD','summer','New','WD ']                       



In [9]:
iter_2_features = iter_1_features + additional_features_iter_2
df = df_clean[iter_2_features]

In [10]:
X = df
y = df_clean['SalePrice']


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42)



In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
lr.score(X_train, y_train)

0.903529635812563

In [14]:
y_train_hat = lr.predict(X_train)

RMSE = np.sqrt(mean_squared_error(y_train, y_train_hat))
RMSE

24518.83497595041

In [15]:
lr.score(X_test, y_test)

0.8841292403236309

In [16]:
cross_val_score(lr, X_train, y_train, cv=5).mean()

0.8857363487412899

In [17]:
# submit to kaggle 
df_kaggle = pd.read_csv('../datasets/test.csv')
df_kaggle.columns
df_kaggle.shape

(879, 80)

In [18]:
def data_clean_kaggle(df):
    # changes due to initial data cleaning
    temp = []
    for col_name in df.columns:
        temp.append(col_name.replace(' ', '_'))
    df.columns = temp

    # changes due to iteration 1
    df['Lot_Frontage_missing'] = df['Lot_Frontage'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Lot_Frontage'].fillna(0, inplace= True)
    
    df['Mas_Vnr_Area_missing'] = df['Mas_Vnr_Area'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Mas_Vnr_Area'].fillna(0, inplace= True)
                                                       
    df['BsmtFin_SF_1_missing'] = df['BsmtFin_SF_1'].isnull().apply(lambda x: 1 if x == True else 0)
    df['BsmtFin_SF_1'].fillna(0, inplace= True)
                                                        
    df['Total_Bsmt_SF_missing'] = df['Total_Bsmt_SF'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Total_Bsmt_SF'].fillna(0, inplace= True)
                              
    df['Bsmt_Full_Bath_missing'] = df['Bsmt_Full_Bath'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Bsmt_Full_Bath'].fillna(0, inplace= True)
                              
    df['Garage_Yr_Blt_missing'] = df['Garage_Yr_Blt'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Garage_Yr_Blt'].fillna(0, inplace= True) 
                              
    df['Garage_Cars_missing'] = df['Garage_Cars'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Garage_Cars'].fillna(0, inplace= True) 
                              
    df['Garage_Area_missing'] = df['Garage_Area'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Garage_Area'].fillna(0, inplace= True)
                              
                            
    df['Bsmt_Qual_missing'] = df['Bsmt_Qual'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Bsmt_Qual'].fillna('NA', inplace= True)
    
    df['Garage_Type_missing'] = df['Garage_Type'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Garage_Type'].fillna('NA', inplace= True) 
    
    # changes due to iteration 2
    df_neighborhood_dummies = pd.get_dummies(df['Neighborhood'], drop_first= True)
    df_bldg_type_dummies = pd.get_dummies(df['Bldg_Type'], drop_first= True)
    df_house_style_dummies = pd.get_dummies(df['House_Style'], drop_first= True)
    def month_to_season_conversion(month):
        if month <= 3:
            return 'spring'
        elif month <= 6:
            return 'summer'
        elif month <= 9:
            return 'fall'
        elif month <= 12:
            return 'winter'
        else:
            print('Houston, we have a problem!')
    df['season'] = df['Mo_Sold'].apply(month_to_season_conversion)
    df_season_dummies = pd.get_dummies(df['season'], drop_first= True)
    df_sale_type_dummies = pd.get_dummies(df['Sale_Type'], drop_first= True)
    df['Bsmt_Cond_missing'] = df['Bsmt_Cond'].isnull().apply(lambda x: 1 if x == True else 0)
    df['Bsmt_Cond'].fillna('NA', inplace= True)

    df = pd.concat([df, df_neighborhood_dummies, df_bldg_type_dummies, df_house_style_dummies, \
               df_season_dummies, df_sale_type_dummies], axis= 1)
    
    
    return df

In [19]:
df_kaggle = data_clean_kaggle(df_kaggle)

X_kaggle = df_kaggle[iter_2_features]

y_kaggle_hat = lr.predict(X_kaggle)

In [20]:
id_list = list(df_kaggle['Id'])
sale_price_list = list(y_kaggle_hat)

df_submission = pd.DataFrame(list(zip(id_list, sale_price_list)),
                                 columns= ['Id', 'SalePrice'])

df_submission.to_csv('../datasets/iter_2_submission.csv', index= False)

In [21]:
model = sm.OLS(y_train, X_train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              SalePrice   R-squared (uncentered):                   0.984
Model:                            OLS   Adj. R-squared (uncentered):              0.983
Method:                 Least Squares   F-statistic:                              1752.
Date:                Wed, 02 Oct 2019   Prob (F-statistic):                        0.00
Time:                        22:59:44   Log-Likelihood:                         -17729.
No. Observations:                1535   AIC:                                  3.556e+04
Df Residuals:                    1483   BIC:                                  3.584e+04
Df Model:                          52                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
PID                  1.063e-05    5.4e-06      1.967      0.049     3.2e-08    2.12e-05
MS_SubClass           -82.5889     43.190     -1.912      0.056    -167.309       2.131
Lot_Frontage           43.5092     22.052      1.973      0.049       0.252      86.766
Lot_Area                0.4745      0.119      3.986      0.000       0.241       0.708
Overall_Qual         1.378e+04    844.126     16.325      0.000    1.21e+04    1.54e+04
Overall_Cond         5814.0040    760.581      7.644      0.000    4322.075    7305.933
Year_Built             49.0544     41.091      1.194      0.233     -31.548     129.657
Year_Remod/Add        -80.2837     40.996     -1.958      0.050    -160.700       0.132
Mas_Vnr_Area           19.8561      4.679      4.243      0.000      10.677      29.035
BsmtFin_SF_1           24.0609      2.260     10.648      0.000      19.628      28.494
Total_Bsmt_SF          27.8118      3.745      7.427      0.000      20.466      35.158
1st_Flr_SF             25.4183     13.524      1.879      0.060      -1.111      51.947
2nd_Flr_SF             30.1468     13.458      2.240      0.025       3.747      56.546
Gr_Liv_Area            23.8795     13.246      1.803      0.072      -2.104      49.863
Bsmt_Full_Bath       3628.0038   1723.427      2.105      0.035     247.391    7008.617
Full_Bath            4915.1876   1992.523      2.467      0.014    1006.725    8823.650
Half_Bath            5816.8130   2014.574      2.887      0.004    1865.095    9768.531
Bedroom_AbvGr       -7891.1324   1271.023     -6.208      0.000   -1.04e+04   -5397.939
TotRms_AbvGrd        1779.7591    884.102      2.013      0.044      45.536    3513.982
Fireplaces           3041.3699   1298.475      2.342      0.019     494.326    5588.414
Garage_Yr_Blt         -13.2612     13.041     -1.017      0.309     -38.843      12.320
Garage_Cars          3616.6414   2260.157      1.600      0.110    -816.802    8050.085
Garage_Area            19.8923      7.803      2.549      0.011       4.587      35.198
Wood_Deck_SF           15.4066      5.937      2.595      0.010       3.761      27.052
Open_Porch_SF          18.1644     11.125      1.633      0.103      -3.658      39.987
Enclosed_Porch        -26.2660     12.206     -2.152      0.032     -50.210      -2.322
Bsmt_Qual_missing    2.852e+04   5954.379      4.790      0.000    1.68e+04    4.02e+04
Garage_Type_missing -1.027e+04   2.54e+04     -0.405      0.686   -6.01e+04    3.95e+04
BrDale               -685.5626   8160.913     -0.084      0.933   -1.67e+04    1.53e+04
BrkSide             -1.049e+04   3811.402     -2.754      0.006    -1.8e+04   -3018.653
Edwards             -1.094e+04   3079.422     -3.553      0.000    -1.7e+04   -4

In [22]:
iter_2_features = ['MS_SubClass', 'Lot_Frontage', 'Lot_Area', 'Overall_Qual',
                        'Overall_Cond', 'BsmtFin_SF_1', 'Total_Bsmt_SF', 'Bsmt_Full_Bath', 
                        'Bedroom_AbvGr', 'TotRms_AbvGrd', 'Fireplaces', 'Garage_Area', 'PID',
                        'Mas_Vnr_Area', '1st_Flr_SF', '2nd_Flr_SF', 'Full_Bath', 'Half_Bath', 'Kitchen_AbvGr', 
                        'Wood_Deck_SF', 'Screen_Porch', 'Bsmt_Qual_missing', 'BrkSide', 'Edwards', 'IDOTRR', 
                        'NAmes', 'NoRidge', 'NridgHt', 'OldTown', 'Somerst', 'StoneBr', '2fmCon', 'New']

In [23]:
df = df_clean[iter_2_features]

In [24]:
X = df
y = df_clean['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42)

In [25]:
lr = LinearRegression()
lr.fit(X_train, y_train)

lr.score(X_train, y_train)

0.8986578387026355

In [26]:
y_train_hat = lr.predict(X_train)

RMSE = np.sqrt(mean_squared_error(y_train, y_train_hat))
RMSE

25130.316175506294

In [27]:
lr.score(X_test, y_test)

0.8798573185558337

In [28]:
cross_val_score(lr, X, y, cv= 5).mean()

0.8889988263874589

In [29]:
# after dropping 25 features from the original iteration 2 due to OLS, R squared dropped 0.007 based on the Cross Varimp

In [30]:
df.to_csv('../datasets/iter_2_df.csv')